# Libraries

In [1]:
import pandas as pd
import numpy as np
import pickle
from pathlib import Path
import dill
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import TargetEncoder
import os
from define_function import *

# Load data

In [2]:
df_test = load_data('test.csv')

In [3]:
id_test = df_test['Id']

# drop features

In [4]:
df_test = drop_features(df_test, features_to_drop=['Alley','PoolQC','Fence','MiscFeature'])

In [5]:
df_test

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,LotShape,LandContour,Utilities,LotConfig,...,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold,SaleType,SaleCondition
0,1461,20,RH,80.0,11622,Pave,Reg,Lvl,AllPub,Inside,...,0,0,0,120,0,0,6,2010,WD,Normal
1,1462,20,RL,81.0,14267,Pave,IR1,Lvl,AllPub,Corner,...,36,0,0,0,0,12500,6,2010,WD,Normal
2,1463,60,RL,74.0,13830,Pave,IR1,Lvl,AllPub,Inside,...,34,0,0,0,0,0,3,2010,WD,Normal
3,1464,60,RL,78.0,9978,Pave,IR1,Lvl,AllPub,Inside,...,36,0,0,0,0,0,6,2010,WD,Normal
4,1465,120,RL,43.0,5005,Pave,IR1,HLS,AllPub,Inside,...,82,0,0,144,0,0,1,2010,WD,Normal
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1454,2915,160,RM,21.0,1936,Pave,Reg,Lvl,AllPub,Inside,...,0,0,0,0,0,0,6,2006,WD,Normal
1455,2916,160,RM,21.0,1894,Pave,Reg,Lvl,AllPub,Inside,...,24,0,0,0,0,0,4,2006,WD,Abnorml
1456,2917,20,RL,160.0,20000,Pave,Reg,Lvl,AllPub,Inside,...,0,0,0,0,0,0,9,2006,WD,Abnorml
1457,2918,85,RL,62.0,10441,Pave,Reg,Lvl,AllPub,Inside,...,32,0,0,0,0,700,7,2006,WD,Normal


# Clean Data

In [6]:
df_test = clean_data(df_test, train=False)

In [7]:
df_test.isna().sum().sum()

0

# Encode Data

In [8]:
Target_Encoding_list = ['MSZoning', 'Street', 'Utilities', 'LotConfig', 'Neighborhood', 'HouseStyle', 'RoofStyle', 'RoofMatl', 'Exterior1st', 'Exterior2nd', 'MasVnrType', 'Foundation', 'BsmtFinType1', 'BsmtFinType2', 'Heating', 'CentralAir', 'Electrical', 'GarageType', 'SaleType']
Ordinal_Encoding_list= ['LotShape', 'LandContour', 'LandSlope', 'Condition1', 'Condition2', 'BldgType', 'ExterQual', 'ExterCond', 'BsmtQual', 'BsmtCond', 'BsmtExposure', 'HeatingQC', 'KitchenQual', 'Functional', 'FireplaceQu', 'GarageFinish', 'GarageQual', 'GarageCond', 'PavedDrive', 'SaleCondition']


encoding_methods = {col: 'target' for col in Target_Encoding_list}
encoding_methods.update({col: 'ordinal' for col in Ordinal_Encoding_list})

df_test = encode_data(df_test, encoding_methods , train=False, target=['SalePrice'])


{'target': TargetEncoder(target_type='continuous'), 'ordinal': OrdinalEncoder()}


In [9]:
df_test

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,LotShape,LandContour,Utilities,LotConfig,...,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold,SaleType,SaleCondition
0,1461.0,20.0,132136.479927,80.0,11622.0,181130.394622,3.0,3.0,180950.936426,176941.547778,...,0.0,0.0,0.0,120.0,0.0,0.0,6.0,2010.0,173406.590852,4.0
1,1462.0,20.0,190995.949254,81.0,14267.0,181130.394622,0.0,3.0,180950.936426,181620.429673,...,36.0,0.0,0.0,0.0,0.0,12500.0,6.0,2010.0,173406.590852,4.0
2,1463.0,60.0,190995.949254,74.0,13830.0,181130.394622,0.0,3.0,180950.936426,176941.547778,...,34.0,0.0,0.0,0.0,0.0,0.0,3.0,2010.0,173406.590852,4.0
3,1464.0,60.0,190995.949254,78.0,9978.0,181130.394622,0.0,3.0,180950.936426,176941.547778,...,36.0,0.0,0.0,0.0,0.0,0.0,6.0,2010.0,173406.590852,4.0
4,1465.0,120.0,190995.949254,43.0,5005.0,181130.394622,0.0,1.0,180950.936426,176941.547778,...,82.0,0.0,0.0,144.0,0.0,0.0,1.0,2010.0,173406.590852,4.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1454,2915.0,160.0,126409.747884,21.0,1936.0,181130.394622,3.0,3.0,180950.936426,176941.547778,...,0.0,0.0,0.0,0.0,0.0,0.0,6.0,2006.0,173406.590852,4.0
1455,2916.0,160.0,126409.747884,21.0,1894.0,181130.394622,3.0,3.0,180950.936426,176941.547778,...,24.0,0.0,0.0,0.0,0.0,0.0,4.0,2006.0,173406.590852,0.0
1456,2917.0,20.0,190995.949254,160.0,20000.0,181130.394622,3.0,3.0,180950.936426,176941.547778,...,0.0,0.0,0.0,0.0,0.0,0.0,9.0,2006.0,173406.590852,0.0
1457,2918.0,85.0,190995.949254,62.0,10441.0,181130.394622,3.0,3.0,180950.936426,176941.547778,...,32.0,0.0,0.0,0.0,0.0,700.0,7.0,2006.0,173406.590852,4.0


# Predict

### RandomForest Model Predictions

In [10]:
with open('trained_model_rf.pickle', 'rb') as f:
    trained_model_rf = dill.load(f)


with open('feature_list.pickle', 'rb') as f:
    train_columns = dill.load(f)


df_test = df_test[train_columns]
for col in train_columns:
    df_test[col] = df_test[col].astype(float)


y_new_pred_rf = predict_model(df_test, trained_model_rf)
y_new_pred_rf = y_new_pred_rf.flatten()

Predictions after inverse transform (if applicable):[125860.37171951 146872.91789266 181843.74102924 ... 154667.79169852
 120938.70947389 210821.00266924]


C:\Users\elige\AppData\Local\Temp\ipykernel_23832\4137934834.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_test[col] = df_test[col].astype(float)
C:\Users\elige\AppData\Local\Temp\ipykernel_23832\4137934834.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_test[col] = df_test[col].astype(float)
C:\Users\elige\AppData\Local\Temp\ipykernel_23832\4137934834.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] 

In [11]:
y_new_pred_rf

array([125860.37171951, 146872.91789266, 181843.74102924, ...,
       154667.79169852, 120938.70947389, 210821.00266924])

### XgbRegressor Model Predictions

In [12]:
with open('trained_model_XG.pickle', 'rb') as f:
    trained_model_XG = dill.load(f)


with open('feature_list.pickle', 'rb') as f:
    train_columns = dill.load(f)

df_test = df_test[train_columns]
for col in train_columns:
    df_test[col] = df_test[col].astype(float)


y_new_pred_XG = predict_model(df_test, trained_model_XG)
y_new_pred_XG = y_new_pred_XG.flatten()

Predictions after inverse transform (if applicable):[124631.52 154569.9  183071.45 ... 161633.5  122293.63 203357.58]


In [13]:
y_new_pred_XG

array([124631.52, 154569.9 , 183071.45, ..., 161633.5 , 122293.63,
       203357.58], dtype=float32)

### Neural Network Model PRedictions

In [14]:
with open('trained_nn_model.pickle', 'rb') as f:
    trained_nn_model = dill.load(f)


y_new_pred_nn = predict_model(df_test, trained_nn_model)
y_new_pred_nn

46/46 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
Inverse transform produced NaNs. Returning raw predictions.
46/46 ━━━━━━━━━━━━━━━━━━━━ 0s 796us/step
Predictions after inverse transform (if applicable):[[154803.3 ]
 [157160.17]
 [215290.06]
 ...
 [156491.62]
 [172144.95]
 [163895.38]]


array([[154803.3 ],
       [157160.17],
       [215290.06],
       ...,
       [156491.62],
       [172144.95],
       [163895.38]], dtype=float32)

In [15]:
y_new_pred_nn.flatten()

array([154803.3 , 157160.17, 215290.06, ..., 156491.62, 172144.95,
       163895.38], dtype=float32)

In [16]:
# Save predictions with corresponding IDs
model_rf = pd.DataFrame({'Id': id_test, 'SalePrice': y_new_pred_rf})
model_rf.to_csv('prediction_rf.csv', index=False)

In [17]:
# Save predictions with corresponding IDs
model_xg = pd.DataFrame({'Id': id_test, 'SalePrice': y_new_pred_XG})
model_xg.to_csv('prediction_xg.csv', index=False)

In [18]:
# Save predictions with corresponding IDs
model_nn = pd.DataFrame({'Id': id_test, 'SalePrice': y_new_pred_nn.flatten()})
model_nn.to_csv('prediction_nn.csv', index=False)